In [5]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
import edward as ed
import pandas as pd
from edward.models import Normal, Poisson, Uniform

In [2]:
data = pd.read_csv("../data/data-attendance-2.txt")

In [6]:
N = len(data)
a_train = [data.A[i] for i in range(N)]
score_train = [data.Score[i] for i in range(N)]
y_train = [data.Y[i] for i in range(N)]

In [10]:
# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

reset_graph()


In [11]:
sess = ed.get_session()

a = tf.placeholder(tf.int32, [N])
score = tf.placeholder(tf.int32, [N])

b1 = Uniform(low=-1000.0, high = 1000.0)
b2 = Uniform(low=-1000.0, high = 1000.0)
b3 = Uniform(low=-1000.0, high = 1000.0)
lamb = tf.exp(b1 + b2 * a + b3* score)
M = Poisson(lamb)

# variational inference
qb1 = Normal(loc=tf.Variable(tf.random_normal([])), 
            scale = tf.nn.softplus(tf.Variable(tf.random_normal([]))))
qb2 = Normal(loc=tf.Variable(tf.random_normal([])), 
            scale = tf.nn.softplus(tf.Variable(tf.random_normal([]))))
qb3 = Normal(loc=tf.Variable(tf.random_normal([])), 
            scale = tf.nn.softplus(tf.Variable(tf.random_normal([]))))

inference = ed.KLqp({b1:qb1, b2:qb2, b3:qb3}, data={a:a_train, score:score_train, M:m_train})
inference.run()

ValueError: Tensor conversion requested dtype float32 for Tensor with dtype int32: 'Tensor("Placeholder:0", shape=(50,), dtype=int32)'